**1. Scraping the table data from wikipedia page using BeautifulSoup and creating a dataframe with only columns**

In [1]:
from bs4 import BeautifulSoup
import requests
import pandas as pd
URL = "https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M"
response = requests.get(URL)
soup = BeautifulSoup(response.text,"html.parser")

table = soup.find('table',{'class':'wikitable sortable'}).tbody

rows = table.find_all('tr')

columns = [v.text.replace('\n','') for v in rows[0].find_all('th')]

df = pd.DataFrame(columns=columns)

In [13]:
df.rename(columns={"Postcode":"PostalCode"},inplace=True)
df.columns

Index(['PostalCode', 'Borough', 'Neighbourhood'], dtype='object')

**2. Reading the row data from table rows and appending it into the df dataframe.**

In [14]:
for i in range(1,len(rows)):
    tds = rows[i].find_all('td')
    if tds[1].text != 'Not assigned':
            if (tds[2].text.replace('\n','') == 'Not assigned'):
                values = [tds[0].text,tds[1].text,tds[1].text]
            else :   
                values = [tds[0].text,tds[1].text,tds[2].text.replace('\n','')]
            df = df.append(pd.Series(values,index=columns),ignore_index=True)
    else:
        values = [td.text.replace('\n','') for td in tds]

**3. Groupby Postcode and Borough and then join by Neighborhood so that there is only row per Postcode and Borough**

In [15]:
df_grouped = df.groupby(['PostalCode','Borough'],as_index=False).agg(lambda group: ','.join(group))

In [16]:
df_grouped

,PostalCode,Borough,Neighbourhood
0,M1B,Scarborough,"Rouge,Malvern,Rouge,Malvern"
1,M1C,Scarborough,"Highland Creek,Rouge Hill,Port Union,Highland ..."
2,M1E,Scarborough,"Guildwood,Morningside,West Hill,Guildwood,Morn..."
3,M1G,Scarborough,"Woburn,Woburn"
4,M1H,Scarborough,"Cedarbrae,Cedarbrae"
5,M1J,Scarborough,"Scarborough Village,Scarborough Village"
6,M1K,Scarborough,"East Birchmount Park,Ionview,Kennedy Park,East..."
7,M1L,Scarborough,"Clairlea,Golden Mile,Oakridge,Clairlea,Golden ..."
8,M1M,Scarborough,"Cliffcrest,Cliffside,Scarborough Village West,..."
9,M1N,Scarborough,"Birch Cliff,Cliffside West,Birch Cliff,Cliffsi..."


In [17]:
df_grouped.shape

(103, 3)

In [18]:
toronto_coordinates = pd.read_csv('C:/Users/dsapre/Documents/GitHub-Projects/MyGitHub-Example/Geospatial_Coordinates.csv')

In [12]:
toronto_coordinates.head()

,Postal Code,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


In [20]:
toronto_coordinates.rename(columns={"Postal Code":"PostalCode"},inplace=True)
toronto_coordinates.columns

Index(['PostalCode', 'Latitude', 'Longitude'], dtype='object')

In [21]:
df_merge = pd.merge(df_grouped,toronto_coordinates, on = 'PostalCode')

In [22]:
df_merge.head()

,PostalCode,Borough,Neighbourhood,Latitude,Longitude
0,M1B,Scarborough,"Rouge,Malvern,Rouge,Malvern",43.806686,-79.194353
1,M1C,Scarborough,"Highland Creek,Rouge Hill,Port Union,Highland ...",43.784535,-79.160497
2,M1E,Scarborough,"Guildwood,Morningside,West Hill,Guildwood,Morn...",43.763573,-79.188711
3,M1G,Scarborough,"Woburn,Woburn",43.770992,-79.216917
4,M1H,Scarborough,"Cedarbrae,Cedarbrae",43.773136,-79.239476


In [23]:
df_merge.shape

(103, 5)